In [1]:
import parameters
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
# start an instance of chrome
driver = webdriver.Chrome("/usr/local/bin/chromedriver")
# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.linkedin.com/company/conquestcyber/people/')

In [4]:
# locate email form by_id
username = driver.find_element_by_id('username')

# send_keys() to simulate key strokes
username.send_keys(parameters.linkedin_username)

# sleep for 0.5 seconds
sleep(0.5)

In [5]:
# locate password form by_id
password = driver.find_element_by_id('password')

# send_keys() to simulate key strokes
password.send_keys(parameters.linkedin_password)
sleep(0.5)

In [6]:
# locate submit button by_xpath
sign_in_button = driver.find_element_by_xpath('//*[@type="submit"]')

# .click() to mimic button click
sign_in_button.click()

In [7]:
# loading all results on the page, clicking hidden "more results button"
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [8]:
# parse the page html
html = driver.page_source
soup_html = soup(html)

In [9]:
#isolate the element holding all of the people list items
people_list_container = soup_html.find("ul", class_="display-flex list-style-none flex-wrap")

In [10]:
# grab all the people(li elements)
people_list = people_list_container.find_all('li')

In [11]:
# check the length looks right
len(people_list)

88

In [12]:
# create pandas df to hold data
ccdb = pd.DataFrame()
ccdb = ccdb.assign(Name = "",Title = "", Company = "")

In [19]:
for person in people_list:
    # find the name text element
    name = person.find("div", class_="org-people-profile-card__profile-title t-black lt-line-clamp lt-line-clamp--single-line ember-view")
    #find the title text element 
    title = person.find("div", class_="lt-line-clamp lt-line-clamp--multi-line ember-view")
    # add name and title to df
    ccdb = ccdb.append({"Name":name.text, "Title":title.text}, ignore_index=True)

AttributeError: 'NoneType' object has no attribute 'text'

In [15]:
#clean up strings (get rid of /n)
ccdb = ccdb.replace(regex = [r'\n', "  "], value='')


In [16]:
ccdb = ccdb.fillna("")
company = ['at Conquest Cyber', '@ Conquest Cyber', 'Conquest Cyber', 'ConQuest Cyber']
company2 = [ 'at UDT', "UDT"]
keywords = ['CISSP', 'CySA+', 'CEH', 'CMMC-AB', 'RP']
for row in range(0, len(ccdb)):
    for string in company:

        if string in str(ccdb.iat[(row), 1]):
            print(ccdb.iat[row, 1])
            ccdb.iat[row, 1] = ccdb.iat[row, 1].replace(string, "")
            print(ccdb.iat[row, 1])
            ccdb.iat[row, 2] = 'Conquest Cyber'
    for string in company2:

        if string in str(ccdb.iat[(row), 1]):
            print(ccdb.iat[row, 1])
            ccdb.iat[row, 1] = ccdb.iat[row, 1].replace(string, "")
            print(ccdb.iat[row, 1])
            ccdb.iat[row, 2] = 'UDT'

CEO at UDT
CEO 
Executive Operations Manager at Conquest Cyber
Executive Operations Manager 
Senior Vice President -Digital Transformation at Conquest Cyber
Senior Vice President -Digital Transformation 
SOC Analyst at Conquest Cyber
SOC Analyst 
Strategy Consultant at Conquest Cyber I CMMC Registered Practitioner
Strategy Consultant  I CMMC Registered Practitioner
Product Development Engineer at Conquest Cyber
Product Development Engineer 
Vice President at Conquest Cyber
Vice President 
Manager, Engagement Management Office at Conquest Cyber
Manager, Engagement Management Office 
SVP of Security Strategy Conquest Cyber
SVP of Security Strategy 
Cyber Threat Intelligence Analyst at Conquest Cyber
Cyber Threat Intelligence Analyst 
Customer Success Manager at Conquest Cyber
Customer Success Manager 
Senior Director of Advanced Technologies at Conquest Cyber
Senior Director of Advanced Technologies 
Engagement Manager at Conquest Cyber
Engagement Manager 
Executive Director of Strategic

In [17]:
ccdb.head()

,Name,Title,Company
0,Henry Fleches,CEO,UDT
1,Carmen Brooks,Executive Operations Manager,Conquest Cyber
2,Benjamin Tosado,Senior Vice President -Digital Transformation,Conquest Cyber
3,Devin Thomas,SOC Analyst,Conquest Cyber
4,Lily Ford,Strategy Consultant I CMMC Registered Practitioner,Conquest Cyber


In [18]:
ccdb.to_csv("conquest_data2.csv", index=False)

In [245]:
ccdb.Title.unique()

array(['CEO ', 'Manager, Engagement Management Office ',
       'Cyber Threat Intelligence Analyst ',
       'Senior Vice President -Digital Transformation ',
       'Executive Operations Manager ',
       'Helping Organizations Achieve CMMC Compliance', 'Vice President ',
       'SVP of Security Strategy ', 'SOC Analyst ',
       'Marketing Specialist ',
       'Strategy Consultant | Helping DoD Contractors with Security Services - CMMC / NIST 800-171 / DFARS',
       'Operations Manager ', 'Engagement Manager ',
       'Cyber Security Analyst', 'SOC Analyst | ', 'Security Analyst ',
       'Ensuring cyber resiliency for the sectors critical to our way of life',
       'Senior Director of Advanced Technologies ',
       'Director of Consulting ', 'Product Development Engineer ',
       'Product Designer ', 'Strategy Consultant ',
       'JR Stratregic Growth Analyst at ConQuest Federal',
       'Software Engineering student', 'User Experience Designer ',
       'Executive Director of 

In [254]:
ccdb["Title"].value_counts().head(20)

Engagement Manager                                                                                                                                         4
Security Engineer                                                                                                                                          3
Security Operations Center Analyst                                                                                                                         3
SOC Analyst                                                                                                                                                2
Strategy Consultant                                                                                                                                        2
Senior SOC Analyst                                                                                                                                         2
MS 365 Solution Architect focusing on Digital Transformati